## **1 кейс**

**Система автопроверки**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить два файла - эталонный код и код пользователя.

In [ ]:
!wget https://gist.github.com/Vs8th/aafed81d81678c807a3ad3dbf93588b2/raw/user.py

!wget https://gist.github.com/Vs8th/95f7897019a4213c76e5b65234d31e30/raw/etalon.py

!wget https://gist.github.com/Vs8th/59f797dfbd33f9a4be1e5ed43bb42f4d/raw/res_cor.txt

--2025-03-24 03:03:56--  https://gist.github.com/Vs8th/aafed81d81678c807a3ad3dbf93588b2/raw/user.py
Resolving gist.github.com (gist.github.com)... 140.82.113.3
Connecting to gist.github.com (gist.github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/aafed81d81678c807a3ad3dbf93588b2/raw/user.py [following]
--2025-03-24 03:03:57--  https://gist.githubusercontent.com/Vs8th/aafed81d81678c807a3ad3dbf93588b2/raw/user.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106 [text/plain]
Saving to: ‘user.py’

user.py             100%[===================>]     106  --.-KB/s    in 0s      

2025-03-24 03:03:57 (2.10 MB/s) - ‘user.py’ saved [106/106]

--2025-03-24 0

Давайте сразу посмотрим с каким кодом мы имеем дело.

In [ ]:
with open('etalon.py', 'r') as f:
    lines = f.readlines()

lines

['def add(a, b):\n', '    return a + b']

In [ ]:
with open('user.py', 'r') as f:
    lines2 = f.readlines()

lines2

['def add_numbers(a, b):\n',
 '    return a - b  # Ошибка: нужно сложить, а не вычесть']

### **Решения**

#### **Задача 1**

Задача не из легких, поэтому начнем по шагам. Для начала напишите функцию `run_tests`, которая прочтет эталонный код и код пользователя из `.py` файлов, выполнит их и сохранит результаты в файл `output.txt`.

**Решение**

Напишите свое решение ниже

In [26]:
from importlib import import_module
from inspect import isfunction, getmembers, getsource
import ast
from multiprocessing import Process

In [24]:
class NoneFunction(Exception):
  pass

In [ ]:
def run_tests(standard_code, user_code, test_cases, output_file):
  """
  Создает файл в формате txt с результатами проверки кода.

    Args:
        standart_code: эталонная модель.
        user_code: модуль для проверки.
        test_cases (list): список данных для тестирования.
        output_file: файл в формате txt с результатами проверки.
  """
  #извлекаем функции из модулей, получаем список ответов из тестовых данных
  function_etalon = getmembers(standard_code, isfunction)
  answers_etalon = [
      func(*test) for _, func in function_etalon
      for test in test_cases
      ]
  function_user = getmembers(user_code, isfunction)
  answers_user = [
      func(*test)
      for _, func in function_user
      for test in test_cases
      ]

  #проверяем файл на системные ошибки
  #если системных ошибок нет, сверяем эталонные ответы с ответами
  #функции студента
  for name, func in function_user:
    if not name:
      check = ['В файле нет функции']
      raise NoneFunction(
          'Ошибка при выполнении кода пользователя: {error}'.format(
              error=check[0]
              )
          )
    else:
      check = list(
        map(
            lambda x, y: 'Тест пройден\n'
            if x == y
            else 'Тест не пройден\nОжидаемый результат: {x}\nРезультат пользователя: {y}\n'.format(
                x=x, y=y
                ),
            answers_etalon, answers_user
            )
        )

  with open('output.txt', 'w') as file:
    file.writelines(check)


# Чтение кодов из файлов
standard_module = import_module('etalon')
user_module = import_module('user')

test_cases = [(1, 2), (3, 4), (5, 6)]
output_file = 'output.txt'
run_tests(standard_module, user_module, test_cases, output_file)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
with open('output.txt', 'r') as file1:
    res1 = file1.read()

with open('res_cor.txt', 'r') as file2:
    res2 = file2.read()

try:
    assert res1 == res2
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 2**

Теперь давайте двигаться дальше. Нужно добавить проверку на вредоносный код. Напишите отдельную функцию - `check_for_malicious_code`, которую мы будем вызывать перед проверкой кода.

Перед написанием кода, выполните следующую ячейку, в ней скачивается новый код пользователя, который проверит работу функции `check_for_malicious_code`.

In [ ]:
!wget https://gist.github.com/Vs8th/9f6a2c73c755cfcc088c7835ec4fb2c0/raw/user_malicious.py

--2025-03-24 03:04:41--  https://gist.github.com/Vs8th/9f6a2c73c755cfcc088c7835ec4fb2c0/raw/user_malicious.py
Resolving gist.github.com (gist.github.com)... 140.82.114.3
Connecting to gist.github.com (gist.github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/9f6a2c73c755cfcc088c7835ec4fb2c0/raw/user_malicious.py [following]
--2025-03-24 03:04:41--  https://gist.githubusercontent.com/Vs8th/9f6a2c73c755cfcc088c7835ec4fb2c0/raw/user_malicious.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117 [text/plain]
Saving to: ‘user_malicious.py’

user_malicious.py   100%[===================>]     117  --.-KB/s    in 0s      

2025-03-24 03:04:41 (5.77 MB/s) - ‘

**Решение**

Напишите свое решение ниже

In [ ]:

FORBIDDEN_NAMES = ['eval', 'exec']

def check_for_malicious_code(code, module, test_cases):
  """
  Проверяет модуль на синтаксические ошибки и наличие вредоносного кода.
  Под вредоносным кодом понимается код, содержаший функции eval и exec

    Args:
        code (str): программный код.
        module: модуль для проверки.
        test_cases (list): список данных для тестирования модуля.

    Returns:
        str: Записи для сохранения в файл в формате txt.
  """
  #проверяем наличие функции в файле
  if not code.startswith('def'):
    check = ['В файле нет функции']
    raise  NoneFunction(
        'Ошибка при выполнении кода пользователя: {error}'.format(
            error=check[0]
            )
        )

  #проверяем синктаксис
  if FORBIDDEN_NAMES[0] not in code and FORBIDDEN_NAMES[1] not in code:
    for _, func in module:
      for test in test_cases:
        try:
          func(*test)
        except Exception as error:
          return 'Код содержит синтаксическую ошибку: {error}'.format(
              error=error
              )

  #проверяем наличие вредоносных функций
  tree = ast.parse(code)
  for node in ast.walk(tree):
    if isinstance(node, ast.Call):
      if isinstance(node.func, ast.Name):
        if node.func.id == FORBIDDEN_NAMES[0]:
          return 'Код содержит вызов {name}()'.format(
              name=node.func.id
              )
        elif node.func.id == FORBIDDEN_NAMES[1]:
          return 'Код содержит запрещенный атрибут: {name}'.format(
              name=node.func.id
              )
  return None


#def run_tests(standard_code, user_code, test_cases, output_file):

    # сюда можете вставить ваш код из прошлой задачи
    #...

# Чтение кодов из файлов
standard_module = import_module('etalon')
user_module = import_module('user')

test_cases = [(1, 2), (3, 4), (5, 6)]
output_file = 'output.txt'
# Проверка на вредоносный код
malicious_code_message = check_for_malicious_code(
    getsource(user_module),
    getmembers(user_module, isfunction),
    test_cases
    )
if malicious_code_message:
    with open(output_file, 'w') as f:
        f.write("Ошибка при проверке модуля user_malicious:\n")
        f.write(malicious_code_message)
else:
    run_tests(standard_module, user_module, test_cases, output_file)


✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/5197125c5ef1e1b34b7d73adccdaf4bb/raw/cor_output_2.txt

--2025-03-24 03:39:53--  https://gist.github.com/Vs8th/5197125c5ef1e1b34b7d73adccdaf4bb/raw/cor_output_2.txt
Resolving gist.github.com (gist.github.com)... 140.82.112.3
Connecting to gist.github.com (gist.github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/5197125c5ef1e1b34b7d73adccdaf4bb/raw/cor_output_2.txt [following]
--2025-03-24 03:39:54--  https://gist.githubusercontent.com/Vs8th/5197125c5ef1e1b34b7d73adccdaf4bb/raw/cor_output_2.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 107 [text/plain]
Saving to: ‘cor_output_2.txt’

cor_output_2.txt    100%[===================>]     107  --.-KB/s    in 0s      

2025-03-24 03:39:54 (1.96 MB/s) - ‘cor_

In [ ]:
with open('output2.txt', 'r') as file1:
    res1 = file1.read()

with open('cor_output_2.txt', 'r') as file2:
    res2 = file2.read()

try:
    assert res1== res2
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 3**

Движемся дальше - последнее, что нам нужно проверить - время выполнения. Если время выполнения превышает 5 секунд, уведомите об этом пользователя и остановите выполнение проверки.

Перед написанием кода, выполните следующую ячейку, в ней скачивается новый код пользователя, который проверит работу проверки на время выполнения.

In [ ]:
!wget https://gist.github.com/Vs8th/4aba49489a8b0843ef85c4a1a93f923d/raw/user_inf.py

--2025-03-24 03:44:33--  https://gist.github.com/Vs8th/4aba49489a8b0843ef85c4a1a93f923d/raw/user_inf.py
Resolving gist.github.com (gist.github.com)... 140.82.114.4
Connecting to gist.github.com (gist.github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/4aba49489a8b0843ef85c4a1a93f923d/raw/user_inf.py [following]
--2025-03-24 03:44:34--  https://gist.githubusercontent.com/Vs8th/4aba49489a8b0843ef85c4a1a93f923d/raw/user_inf.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55 [text/plain]
Saving to: ‘user_inf.py’

user_inf.py         100%[===================>]      55  --.-KB/s    in 0s      

2025-03-24 03:44:34 (2.36 MB/s) - ‘user_inf.py’ saved [55/55

**Решение**

Напишите свое решение ниже

In [36]:

FORBIDDEN_NAMES = ['eval']

def check_for_malicious_code(code, module, test_cases):
  """
  Проверяет модуль на синтаксические ошибки и наличие вредоносного кода.
  Под вредоносным кодом понимается код, содержаший функцию eval.

    Args:
        code (str): программный код.
        module: модуль для проверки.
        test_cases (list): список данных для тестирования модуля.

    Returns:
        str: Записи для сохранения в файл в формате txt.
  """
  #проверяем наличие функции в файле
  if not code.startswith('def'):
    check = ['В файле нет функции']
    raise  NoneFunction(
        'Ошибка при выполнении кода пользователя: {error}'.format(
            error=check[0]
            )
        )

  #проверяем синктаксис
  if FORBIDDEN_NAMES[0] not in code:
    for _, func in module:
      for test in test_cases:
        try:
          process = Process(target=func, args=test)
          process.start()
          process.join(timeout=5)
          if process.is_alive():
            return 'Предупреждение: время выполнения превышает 5 секунд.'
        except Exception as error:
          return 'Код содержит синтаксическую ошибку: {error}'.format(
              error=error
              )

  #проверяем наличие вредоносных функций
  tree = ast.parse(code)
  for node in ast.walk(tree):
    if isinstance(node, ast.Call):
      if isinstance(node.func, ast.Name):
        if node.func.id == FORBIDDEN_NAMES[0]:
          return 'Код содержит вызов {name}()'.format(
              name=node.func.id
              )

  return None

#def run_tests(standard_code, user_code, test_cases, output_file):

    # сюда можете вставить ваш код из прошлой задачи
    # в него необходимо будет добавить проверку на время выполнения
   #...


# Чтение кодов из файлов
standard_module = import_module('etalon')
user_module = import_module('user_inf')

test_cases = [(1, 2), (3, 4), (5, 6)]
output_file = 'output3.txt'
# Проверка на вредоносный код
malicious_code_message = check_for_malicious_code(
      getsource(user_module),
      getmembers(user_module, isfunction),
      test_cases
  )

if malicious_code_message:
    with open(output_file, 'w') as f:
       # f.write("Ошибка при проверке модуля user_malicious:\n")
        f.write(malicious_code_message)
else:
    run_tests(standard_module, user_module, test_cases, output_file)


✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [33]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/cfcafbd2cd46e8905555b02eae78406e/raw/cor_res_3.txt

--2025-03-24 04:59:10--  https://gist.github.com/Vs8th/cfcafbd2cd46e8905555b02eae78406e/raw/cor_res_3.txt
Resolving gist.github.com (gist.github.com)... 140.82.113.3
Connecting to gist.github.com (gist.github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/cfcafbd2cd46e8905555b02eae78406e/raw/cor_res_3.txt [following]
--2025-03-24 04:59:10--  https://gist.githubusercontent.com/Vs8th/cfcafbd2cd46e8905555b02eae78406e/raw/cor_res_3.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96 [text/plain]
Saving to: ‘cor_res_3.txt’

cor_res_3.txt       100%[===================>]      96  --.-KB/s    in 0s      

2025-03-24 04:59:10 (5.05 MB/s) - ‘cor_res_3.txt’ sa

In [37]:
with open('output3.txt', 'r') as file1:
    res1 = file1.read()

with open('cor_res_3.txt', 'r') as file2:
    res2 = file2.read()

try:
    assert res1 == res2
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!
